In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


In [2]:
from src.speech.text_to_speech import SpeechSynthesizer
from src.speech.speech_recognizer import StreamingSpeechRecognizerFromBytes
from openai import AzureOpenAI

if "az_speech_recognizer_stream_client" not in locals():
    az_speech_recognizer_stream_client = StreamingSpeechRecognizerFromBytes(
        vad_silence_timeout_ms=800,
        use_semantic_segmentation=False,
        audio_format="pcm",
        candidate_languages=["en-US", "fr-FR", "de-DE", "es-ES", "it-IT"],
        enable_diarisation=True,
        speaker_count_hint=2,
        enable_neural_fe=False,
    )

if "az_speach_synthesizer_client" not in locals():
    az_speach_synthesizer_client = SpeechSynthesizer()

# Ensure Azure OpenAI client is initialized only if not already defined
if "client" not in locals():
    client = AzureOpenAI(
        api_version="2025-02-01-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    )

[2025-08-06 10:44:02,823] INFO - micro: Azure Monitor tracing initialized for speech recognizer
INFO:micro:Azure Monitor tracing initialized for speech recognizer
[2025-08-06 10:44:02,835] INFO - micro: Creating SpeechConfig with API key authentication
INFO:micro:Creating SpeechConfig with API key authentication
[2025-08-06 10:44:02,850] INFO - micro: Azure Monitor tracing initialized for speech synthesizer
INFO:micro:Azure Monitor tracing initialized for speech synthesizer
[2025-08-06 10:44:02,862] INFO - micro: Creating SpeechConfig with API key authentication
INFO:micro:Creating SpeechConfig with API key authentication
[2025-08-06 10:44:02,878] INFO - micro: Speech synthesizer initialized successfully
INFO:micro:Speech synthesizer initialized successfully


In [3]:
# Define end-of-sentence markers for TTS
TTS_ENDS = {".", "!", "?", ";", "\n"}

# Flags and buffers
is_synthesizing = False
user_buffer = ""
assistant_buffer = ""
tts_thread = None

In [ ]:
import os, time, threading


def on_final(text, lang):
    global user_buffer
    user_buffer += text.strip() + "\n"
    print(f"🧾 User (final) in {lang}: {text}")


def assistant_speak(text):
    global is_synthesizing
    print("Hi there, I am a assistant_speak callback!")
    is_synthesizing = True
    print("Syntetixing:", is_synthesizing)
    az_speach_synthesizer_client.start_speaking_text(text)


def on_partial(text, lang):
    global is_synthesizing
    if is_synthesizing:
        # az_speach_synthesizer_client.stop_speaking()
        is_synthesizing = False
    print(f"🗣️ User (partial) in {lang}: {text}")


az_speech_recognizer_stream_client.set_partial_result_callback(on_partial)
az_speech_recognizer_stream_client.set_final_result_callback(on_final)

# Start recognition
az_speech_recognizer_stream_client.start()
print("🎙️ Speak now...")

# Start mic streaming thread
import pyaudio

RATE, CHANNELS, CHUNK = 16000, 1, 1024
audio = pyaudio.PyAudio()
stream = audio.open(
    format=pyaudio.paInt16,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK,
)


def mic_loop():
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        az_speech_recognizer_stream_client.write_bytes(data)


threading.Thread(target=mic_loop, daemon=True).start()

messages = [{"role": "system", "content": "You are a helpful assistant."}]

user_buffer = ""  # This should be filled in by your STT callback as before

while True:
    if user_buffer:
        full_conversation = (
            "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
            + f"\nUser: {user_buffer}"
        )
        messages.append({"role": "user", "content": user_buffer})
        user_buffer = ""  # clear after using

        completion = client.chat.completions.create(
            stream=True,
            messages=messages,
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
            model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
        )

        collected_messages = []
        last_tts_request = None

        for chunk in completion:
            if chunk.choices and hasattr(chunk.choices[0].delta, "content"):
                chunk_text = chunk.choices[0].delta.content
                if chunk_text:
                    collected_messages.append(chunk_text)
                    print(chunk_text, end="", flush=True)
                    # Check for sentence end to stream to TTS
                    if chunk_text in TTS_ENDS:
                        text = "".join(collected_messages).strip()
                        last_tts_request = assistant_speak(text)
                        collected_messages.clear()
        print()  # finish line after streaming LLM response

    time.sleep(0.1)

[2025-08-06 10:44:07,077] INFO - micro: Starting recognition from byte stream...
INFO:micro:Starting recognition from byte stream...
[2025-08-06 10:44:07,094] INFO - micro: Recognition started.
INFO:micro:Recognition started.


🎙️ Speak now...
🗣️ User (partial) in en-US: yes i understand that
🗣️ User (partial) in en-US: yes i understand that and
🗣️ User (partial) in en-US: yes i understand that and i
🗣️ User (partial) in en-US: yes i understand that and i agree
🗣️ User (partial) in en-US: yes i understand that and i agree with
🗣️ User (partial) in en-US: yes i understand that and i agree with that
🗣️ User (partial) in es-ES: grande stan
🗣️ User (partial) in es-ES: grande standard
🗣️ User (partial) in es-ES: sean standard and i
🗣️ User (partial) in es-ES: sean standard and i greewichet
🗣️ User (partial) in es-ES: sean standard and i greewichet an
🗣️ User (partial) in es-ES: sean standard and i greewichet **** just in
🗣️ User (partial) in es-ES: sean standard and i greewichet **** just in to the
🗣️ User (partial) in es-ES: sean standard and i greewichet **** just in to the re
🗣️ User (partial) in es-ES: sean standard and i greewichet **** just in to the really things
🗣️ User (partial) in es-ES: sean standard an

[2025-08-06 10:44:18,128] INFO - micro: [🔊] Starting streaming speech synthesis for text: It seems like your input may be incomplete or uncl...
INFO:micro:[🔊] Starting streaming speech synthesis for text: It seems like your input may be incomplete or uncl...


 Are you looking for information about Sean Standard and Greenwich?Hi there, I am a assistant_speak callback!
Syntetixing: True


[2025-08-06 10:44:18,210] INFO - micro: [🔊] Starting streaming speech synthesis for text: Are you looking for information about Sean Standar...
INFO:micro:[🔊] Starting streaming speech synthesis for text: Are you looking for information about Sean Standar...


...

[2025-08-06 10:44:19,703] INFO - micro: [🔊] Starting streaming speech synthesis for text: If you'd like to share more details or if there's ...
INFO:micro:[🔊] Starting streaming speech synthesis for text: If you'd like to share more details or if there's ...


 I'm here to help you think it through!Hi there, I am a assistant_speak callback!
Syntetixing: True


[2025-08-06 10:44:19,764] INFO - micro: [🔊] Starting streaming speech synthesis for text: I'm here to help you think it through!...
INFO:micro:[🔊] Starting streaming speech synthesis for text: I'm here to help you think it through!...


KeyboardInterrupt: 

🗣️ User (partial) in en-US: OK
🗣️ User (partial) in en-US: OK so but the
🗣️ User (partial) in en-US: OK so but the problem that
🗣️ User (partial) in en-US: OK so but the problem that i'm trying
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not as
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not as easy
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not as easy you know
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not as easy you know like
🗣️ User (partial) in en-US: OK so but the problem that i'm trying to set is that is not as

In [ ]:
az_speech_recognizer_stream_client.close_stream()
az_speach_synthesizer_client.stop_speaking()

[2025-08-06 10:45:58,878] INFO - micro: [🛑] Stopping speech synthesis...
INFO:micro:[🛑] Stopping speech synthesis...


[2025-08-06 10:45:59,049] WARNING - micro: Recognition canceled: SpeechRecognitionCanceledEventArgs(session_id=9993690b4d444f6e96612b9f9d8e3a57, result=SpeechRecognitionResult(result_id=dcace2598fb847c484b5fe936d86722d, text="", reason=ResultReason.Canceled))
[2025-08-06 10:45:59,067] WARNING - micro: Reason: CancellationReason.EndOfStream, Error: 
[2025-08-06 10:45:59,080] INFO - micro: Session stopped.
INFO:micro:Session stopped.
